In [1]:
pip install torch transformers sentencepiece


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import tkinter as tk
from tkinter import ttk, messagebox
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# === Load model and tokenizer from local path ===
MODEL_PATH = "./nllb_distilled_model"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

# === Language code mapping ===
LANGUAGES = {
    "English": "eng_Latn",
    "French": "fra_Latn",
    "Hindi": "hin_Deva",
    "Spanish": "spa_Latn",
    "Bengali": "ben_Beng",
    "Tamil": "tam_Taml",
    "Telugu": "tel_Telu",
    "Arabic": "arb_Arab",
    "Russian": "rus_Cyrl",
    "Chinese": "zho_Hans",
    "Japanese": "jpn_Jpan"
}

# === Translation function ===
def translate(text, src_lang_code, tgt_lang_code):
    try:
        tokenizer.src_lang = src_lang_code
        encoded = tokenizer(text, return_tensors="pt")
        forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang_code)
        output = model.generate(**encoded, forced_bos_token_id=forced_bos_token_id)
        translated = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        return translated
    except Exception as e:
        return f"❌ Error: {str(e)}"

# === GUI setup ===
def create_gui():
    root = tk.Tk()
    root.title("🌐 Offline Translator - NLLB 200")
    root.geometry("800x500")
    root.configure(bg="#E3F2FD")
    root.resizable(False, False)

    # === Styles ===
    style = ttk.Style()
    style.theme_use("clam")
    style.configure("TLabel", background="#E3F2FD", font=("Segoe UI", 11))
    style.configure("TButton", font=("Segoe UI", 11, "bold"), foreground="white", background="#1565C0")
    style.map("TButton", background=[("active", "#1976D2")])
    style.configure("TCombobox", font=("Segoe UI", 11))

    # === Header ===
    header = tk.Label(root, text="🌍 Multilingual Translator", font=("Segoe UI", 18, "bold"), bg="#1976D2", fg="white")
    header.pack(fill="x", pady=(0, 10))

    # === Input Text ===
    tk.Label(root, text="Enter Text:", font=("Segoe UI", 12, "bold")).pack(anchor="w", padx=15, pady=(5, 0))
    input_text = tk.Text(root, height=5, font=("Segoe UI", 12), wrap="word", bg="#FFFFFF", relief="groove", bd=2)
    input_text.pack(fill="x", padx=15, pady=(0, 10))

    # === Language Selection ===
    frame = tk.Frame(root, bg="#E3F2FD")
    frame.pack(pady=10)

    src_var = tk.StringVar(value="French")
    tgt_var = tk.StringVar(value="English")

    ttk.Label(frame, text="Translate From:").grid(row=0, column=0, padx=10)
    src_menu = ttk.Combobox(frame, textvariable=src_var, values=list(LANGUAGES.keys()), state="readonly", width=20)
    src_menu.grid(row=0, column=1, padx=5)

    ttk.Label(frame, text="To:").grid(row=0, column=2, padx=10)
    tgt_menu = ttk.Combobox(frame, textvariable=tgt_var, values=list(LANGUAGES.keys()), state="readonly", width=20)
    tgt_menu.grid(row=0, column=3, padx=5)

    # === Translate Button ===
    def on_translate():
        text = input_text.get("1.0", tk.END).strip()
        if not text:
            messagebox.showwarning("Input Required", "Please enter some text to translate.")
            return
        src_code = LANGUAGES[src_var.get()]
        tgt_code = LANGUAGES[tgt_var.get()]
        result = translate(text, src_code, tgt_code)
        output_text.delete("1.0", tk.END)
        output_text.insert(tk.END, result)

    translate_btn = ttk.Button(root, text="🔁 Translate", command=on_translate)
    translate_btn.pack(pady=10)

    # === Output Text ===
    tk.Label(root, text="Translation:", font=("Segoe UI", 12, "bold")).pack(anchor="w", padx=15, pady=(5, 0))
    output_text = tk.Text(root, height=5, font=("Segoe UI", 12), wrap="word", bg="#F1F8E9", relief="groove", bd=2)
    output_text.pack(fill="x", padx=15, pady=(0, 10))

    root.mainloop()

create_gui()
